In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 12:50:11.644413: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:50:11.674505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:50:11.675329: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 12:50:12.497327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
1020


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0511                 258.312  -     20   +     21       False
background_yield_0.0511             961.517  -     33   +     34       False
frac_Mbc_bkg1_0.0511               0.593647  -  0.047   +  0.046       False
frac_deltaE_bkg1_0.0511            0.543068  -    0.2   +    0.2       False
True
Iteration 1 - Signal Yield: 258.31203900198255 ± 20.646190971410256
Running iteration 2/300...
200
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1033                 249.021  -     20   +     21       False
background_yield_0.1033             970.885  -     34   +     34       False
frac_Mbc_bkg1_0.1033               0.605576  -  0.047   +  0.046       False
frac_deltaE_bkg1_0.1033            0.516262 

## 150

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9532                 202.883  -     19   +     19       False
background_yield_0.9532             966.994  -     33   +     34       False
frac_Mbc_bkg1_0.9532               0.601806  -  0.047   +  0.045       False
frac_deltaE_bkg1_0.9532            0.563397  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 202.8831466620968 ± 19.00916034545088
Running iteration 2/300...
150
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6079                 200.542  -     19   +     19       False
background_yield_0.6079             969.396  -     33   +     34       False
frac_Mbc_bkg1_0.6079               0.603257  -  0.046   +  0.046       False
frac_delta

## 100

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8065                  147.32  -     17   +     18       False
background_yield_0.8065             972.301  -     33   +     34       False
frac_Mbc_bkg1_0.8065               0.608874  -  0.046   +  0.045       False
frac_deltaE_bkg1_0.8065            0.572171  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 147.3204049519286 ± 17.15526352219443
Running iteration 2/300...
100
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9868                 142.408  -     17   +     17       False
background_yield_0.9868              977.62  -     33   +     34       False
frac_Mbc_bkg1_0.9868                0.61344  -  0.045   +  0.046       False
frac_delta

## 75

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9985                 119.605  -     16   +     17       False
background_yield_0.9985             975.418  -     33   +     34       False
frac_Mbc_bkg1_0.9985                0.61265  -  0.045   +  0.045       False
frac_deltaE_bkg1_0.9985            0.582316  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 119.60477656255883 ± 16.176468264044786
Running iteration 2/300...
75
1020
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.204                 116.462  -     16   +     17       False
background_yield_0.204             978.264  -     33   +     34       False
frac_Mbc_bkg1_0.204               0.616817  -  0.045   +  0.045       False
frac_deltaE_bkg

## 50

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4634                 89.1345  -     15   +     15       False
background_yield_0.4634             980.775  -     33   +     34       False
frac_Mbc_bkg1_0.4634               0.620416  -  0.045   +  0.044       False
frac_deltaE_bkg1_0.4634            0.585274  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 89.13444626923841 ± 14.9095563260294
Running iteration 2/300...
50
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3291                 87.8813  -     15   +     15       False
background_yield_0.3291             981.864  -     33   +     34       False
frac_Mbc_bkg1_0.3291               0.622239  -  0.044   +  0.045       False
frac_deltaE_b

frac_Mbc_bkg1_0.6665               0.310272  -  0.036   +  0.036       False
frac_deltaE_bkg1_0.6665            0.055113  -   0.13   +   0.13       False
True
Iteration 96 - Signal Yield: 2433.3246713255635 ± 53.85116435998698
Running iteration 97/100...
2381.8491017785136
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3964                 2471.24  -     54   +     55       False
background_yield_0.3964             1980.13  -     49   +     50       False
frac_Mbc_bkg1_0.3964               0.290972  -  0.037   +  0.036       False
frac_deltaE_bkg1_0.3964           0.0287554  -   0.13   +   0.13       False
True
Iteration 97 - Signal Yield: 2471.2435562409755 ± 54.14350429115731
Running iteration 98/100...
2391.0938153673433
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------

## 25

In [10]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8759                 70.4928  -     13   +     14       False
background_yield_0.8759             974.499  -     33   +     33       False
frac_Mbc_bkg1_0.8759                 0.6099  -  0.045   +  0.045       False
frac_deltaE_bkg1_0.8759            0.551065  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 70.49276692956826 ± 13.7497448392292
Running iteration 2/300...
25
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8032                 63.6332  -     13   +     14       False
background_yield_0.8032             981.377  -     33   +     34       False
frac_Mbc_bkg1_0.8032               0.620002  -  0.044   +  0.044       False
frac_deltaE_b

## 10

In [11]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7512                 52.1098  -     12   +     13       False
background_yield_0.7512             976.738  -     32   +     35       False
frac_Mbc_bkg1_0.7512               0.614194  -  0.045   +  0.044       False
frac_deltaE_bkg1_0.7512            0.565281  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 52.1097716857996 ± 12.787095131648197
Running iteration 2/300...
10
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5714                 46.0567  -     12   +     13       False
background_yield_0.5714             983.304  -     32   +     34       False
frac_Mbc_bkg1_0.5714               0.624098  -  0.044   +  0.044       False
frac_deltaE_

## 5

In [12]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2283                 47.1077  -     12   +     13       False
background_yield_0.2283             977.908  -     33   +     33       False
frac_Mbc_bkg1_0.2283               0.614085  -  0.045   +  0.044       False
frac_deltaE_bkg1_0.2283            0.570412  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 47.1077305708833 ± 12.520888431275061
Running iteration 2/300...
5
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4228                 42.2373  -     12   +     13       False
background_yield_0.4228             982.795  -     33   +     33       False
frac_Mbc_bkg1_0.4228               0.622497  -  0.044   +  0.044       False
frac_deltaE_bk

frac_Mbc_bkg1_0.7276               0.354897  -  0.032   +  0.032       False
frac_deltaE_bkg1_0.7276            0.168362  -   0.12   +   0.12       False
True
Iteration 96 - Signal Yield: 517.791784443369 ± 28.100793762199558
Running iteration 97/100...
471.5720563185149
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1195                 520.234  -     28   +     28       False
background_yield_0.1195             2020.95  -     47   +     48       False
frac_Mbc_bkg1_0.1195               0.348573  -  0.032   +  0.032       False
frac_deltaE_bkg1_0.1195            0.193819  -   0.12   +   0.13       False
True
Iteration 97 - Signal Yield: 520.2343523497123 ± 28.04466287834341
Running iteration 98/100...
480.2083299603238
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ----------

## 2

In [14]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1742                 43.5615  -     12   +     13       False
background_yield_0.1742             978.473  -     33   +     33       False
frac_Mbc_bkg1_0.1742                0.61533  -  0.045   +  0.044       False
frac_deltaE_bkg1_0.1742            0.573085  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 43.561506875513956 ± 12.313540800164304
Running iteration 2/300...
2
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0269                 40.5586  -     12   +     13       False
background_yield_0.0269              981.35  -     33   +     34       False
frac_Mbc_bkg1_0.0269               0.620059  -  0.044   +  0.044       False
frac_deltaE_

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 4, 'rec511', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg4_decay_ee_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg4_decay_ee_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
1020


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4784                 39.6492  -     12   +     13       False
background_yield_0.4784             980.316  -     33   +     33       False
frac_Mbc_bkg1_0.4784               0.618249  -  0.045   +  0.043       False
frac_deltaE_bkg1_0.4784            0.579022  -   0.18   +   0.19       False
True
Iteration 1 - Signal Yield: 39.649198273228 ± 12.152970985615934
Running iteration 2/300...
0
1020
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6179                 39.6492  -     12   +     13       False
background_yield_0.6179             980.316  -     33   +     33       False
frac_Mbc_bkg1_0.6179               0.618249  -  0.045   +  0.043       False
frac_deltaE_bkg1_0.6179            0.579022  -   